<a href="https://colab.research.google.com/github/catawba-data-mining/CIS-3902-Data-Mining/blob/main/homework7_decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees, Random Forests and the Confusion Matrix for Evaluation
## Data Mining Catawba College
## Group Members:
## 1.
## 2.



# Directions: Open in COLAB.  Carefully study the documentation and execute the code.  Answer the questions throughout the lab and at the end of the lab. They are labeled "Question 1" etc. Save your work and turn in the .ipynb file.

For this project we will be exploring publicly available data from [LendingClub.com](www.lendingclub.com). Lending Club connects people who need money (borrowers) with people who have money (investors). Hopefully, as an investor you would want to invest in people who showed a profile of having a high probability of paying you back. We will try to create a model that will help predict this.

### Research Goal:  Develop a predictive model using decision trees to determine applicants that are most likely to be able to pay back their loans.

Lending club had a [very interesting year in 2016](https://en.wikipedia.org/wiki/Lending_Club#2016), so let's check out some of their data and keep the context in mind. This data is from before they even went public.

We will use lending data from 2007-2010 and be trying to classify and predict whether or not the borrower paid back their loan in full. You can download the original data from [here](https://www.lendingclub.com/info/download-data.action) but we are using a csv provided as it has been cleaned of NA values or NULL values.

### Data Understanding - Here are what the columns represent:
* int.rate: The interest rate of the loan, as a proportion (a rate of 11% would be stored as 0.11). Borrowers judged by LendingClub.com to be more risky are assigned higher interest rates.
* installment: The monthly installments owed by the borrower if the loan is funded.
* log.annual.inc: The natural log of the self-reported annual income of the borrower.
* dti: The debt-to-income ratio of the borrower (amount of debt divided by annual income).
* fico: The FICO credit score of the borrower.
* days.with.cr.line: The number of days the borrower has had a credit line.
* revol.bal: The borrower's revolving balance (amount unpaid at the end of the credit card billing cycle).
* revol.util: The borrower's revolving line utilization rate (the amount of the credit line used relative to total credit available).
* inq.last.6mths: The borrower's number of inquiries by creditors in the last 6 months.
* pub.rec: The borrower's number of derogatory public records (bankruptcy filings, tax liens, or judgments).
* fully.paid - yes or 1 if the loan was paid back in full (yes is good), no or 0  means it was not paid back in full

# 1.  Import Libraries

**Import the usual libraries for pandas and plotting. You can import sklearn later on.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 2. Get the Data

** Use pandas to read loan_data.csv as a dataframe called loans.  Remember this file has the NULL values removed. **

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/catawba-data-mining/CIS-3902-Data-Mining/main/loan_data_prep.csv')

## 3. Look at the first few records, the attributes, and attributes types.
** Check out the info(), head(), and describe() methods on loans.**

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

##  4.  Exploratory Data Analysis:  Visualization

Let's do some data visualization!

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df[df['credit.policy']==1]['fico'],color='blue',bins=30,alpha=0.5,edgecolor='blue')
plt.hist(df[df['credit.policy']==0]['fico'],color='red',bins=30,alpha=0.5,edgecolor='red')
plt.legend(['credit.policy=1','credit.policy=0'])
plt.xlabel("FICO")


## QUESTION 1:  (open up the markdown cell and key in your answer).  What does the above chart show about the relationship of FICO score and Credit Policy?<br>


** Create a histogram -  select by the not.fully.paid column.**<br>
** fully paid = 1 is good **

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df[df['fully.paid']==1]['fico'],color='red',bins=30,alpha=0.5,edgecolor='red')
plt.hist(df[df['fully.paid']==0]['fico'],color='blue',bins=30,alpha=0.5,edgecolor='blue')
plt.legend(['fully.paid=1','fully.paid=0'])
plt.xlabel("FICO")

## QUESTION 2: What does the above chart show about the relationship of FICO score and Not Fully Paid?<br>


** Let's see the trend between FICO score and interest rate.**

In [ ]:
sns.jointplot('fico','int.rate',data=df,color='purple',size=8,ratio=3,space=0.4)

## QUESTION 3: In the chart above, do you see a trend between FICO score and Interest Rate?<br>


** Create the following lmplots to see if the trend differed between not.fully.paid and credit.policy.**

In [ ]:
sns.lmplot('fico','int.rate',data=df,hue='credit.policy',col='fully.paid',size=8)

## QUESTION 4: What do the above two charts show regarding the trend between FICO score and Interest Rate, and how it may or may not differ depending on the "Not Fully Paid" Attribute? <br>


## 5.  Data Preparation

Let's get our data ready for decision trees and random forest modeling.  <br><br>
A decision tree is a flowchart-like tree structure where an internal node represents feature(or attribute), the branch represents a decision rule, and each leaf node represents the outcome. The topmost node in a decision tree is known as the root node. It learns to partition on the basis of the attribute value. It partitions the tree in recursively manner call recursive partitioning. This flowchart-like structure helps you in decision making. It's visualization like a flowchart diagram which easily mimics the human level thinking. That is why decision trees are easy to understand and interpret.<br><br>
Random Forests are like a collection of decision trees taken from random samples from the data:
It works in four steps:<br>   Select random samples from a given dataset.<br>
    Construct a decision tree for each sample and get a prediction result from each decision tree.<br>
    Perform a vote for each predicted result.<br>
    Select the prediction result with the most votes as the final prediction.<br><br>
Random Forests almost always outperform Decision Trees.


** Let's start by checking loans.info() again.**

In [ ]:
df.info()

### We are using Python's scikit-learn to make our modeling process easier with this powerful library.<br><br>
<p>It is important to understand how to prepare data for each type of modeling.</p>
<p> We have a target variable, Not Fully Paid, to predict if we are going to get paid back or not. Since we have a target variable this is Supervised learning.</p><p>We have already removed nulls, this is generally important although just simply removing them isn't always the best practice because you might remove too much of the data!  More on this later.  </p>
<p>Decision trees handle numerical values and categorical values, although scikit-learn requires us to change the categorical variables to "dummy" variables.  This means if we have a Cateorical Variable Direction with North, South, East and West as the possible categories, this will change to three variables (not four) North, South, and East.  If the direction for a particular record was "North", then North will have "1" and the rest will have "0". If the direction is "West" then the three will have 0 . . . all 0's mean the last category or West.</p>
<p>The original data set has a categorical feature "Loan Purpose" which has been removed from the data due to its lack of importance in the prediction</p>
<p> Numerical features for decision trees do not have to be "normalized" meaning if you have numerical variables in different ranges, you do not have to convert them to a similar scale.</p>


### Train and Test.  We train our model on the training data, and test the accuracy of the model we have built on test.

Now its time to split our data into a training set and a testing set!

In [ ]:
# our target is the fully paid variable
# the train set will be trained on the target so we can learn what it means
# again this is supervised learning
# we will test our model on the test set
X = df.drop('fully.paid',axis=1)
y = df['fully.paid']

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [ ]:
# train will have 70% of the data, test will have 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

## 6.  Modeling:  Training a Decision Tree Model

Let's start by training a single decision tree first!

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
dTree = DecisionTreeClassifier()

In [ ]:
dTree.fit(X_train,y_train)

In [ ]:
# this may take a few minutes to run
tree.plot_tree(dTree) 

In [ ]:
#Lets take a look 
#this also takes a minute . . . or two
text_representation = tree.export_text(dTree)
print(text_representation)

In [ ]:
# another way to visualize the tree
# this tree has lots of branches
fn=['credit.policy','int.rate','installment','log.annual.inc','dti','fico','days.with.cr.line','revol.bal','revol.util','inq.last.6mths']
cn=['0','1']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (14,14), dpi=300)
tree.plot_tree(dTree,
               feature_names = fn, 
               class_names=cn,
               filled = True);fig.savefig('imagename.png')

## 7. Predictions and Evaluation of Decision Tree
**Create predictions from the test set and create a confusion matrix and classfication report to evaluate our model.**

In [ ]:
# create the predictions from the test set
predictions = dTree.predict(X_test)

In [ ]:
# import the necessary libraries from sklearn.metrics
from sklearn.metrics import classification_report,confusion_matrix

## Evaluation using a confusion matrix
A confusion matrix is a great way to evaluate a classifcation model.<br>

In [ ]:
#create the confusion matrix
cf_matrix = confusion_matrix(y_test,predictions)
print(cf_matrix)
# Can we look at this in a better, more intuitive way? What does this mean?

In [ ]:
# use seaborn to make the confusion matrix more intuitive
# True Positives are Predicted Yes, Actual Yes or 2019
# True Negatives are Predicted No, Actual No or 93
# False Positives are Predicted Yes, Actual No or 350
# False Negatives are Predicted No, Actual Yes or 412
# Total = 2019+93+350+412 = 2874
# Remember this is the test set - 30% of total records

xlabels = ['Predicted: No','Predicted: Yes']
ylabels = ['Actual: No', 'Actual: Yes']
sns.heatmap(cf_matrix,xticklabels=xlabels,yticklabels=ylabels,annot=True,fmt='g',cmap='Blues')

In [ ]:
# Let's see the percentages in each column, this will help us understand the classification report
xlabels = ['Predicted: No','Predicted: Yes']
ylabels = ['Actual: No', 'Actual: Yes']
sns.heatmap(cf_matrix/np.sum(cf_matrix),xticklabels=xlabels,yticklabels=ylabels,annot=True,fmt='.2%',cmap='Blues')

## Here is the classification report built from the confusion matrix<br>

Learn more about the numbers here and see question below that you need to answer:  <a href="https://www.analyticsvidhya.com/blog/2020/04/confusion-matrix-machine-learning/">Explaining the Confusion Matrix</a>

In [ ]:
#print the classification report
print(classification_report(y_test,predictions))

## Question 5:  Using the link provided above, explain the classification report and confusion matrix charts that have been built. Is the F1 score the best measure of accuracy?

## 8. Training the Random Forest model

Now its time to train our Random Forest Model which will built lots of decision trees and average the results.  This is called an ensemble model and is usually a great model for data that has a target variable (supervised) and can benefit from decision tree analysis!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=300)

In [ ]:
rfc.fit(X_train,y_train)

## Predictions and Evaluation

Let's predict off the y_test values and evaluate our model.

** Predict the class of fully.paid for the X_test data.**

In [ ]:
rfc_pred = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test,rfc_pred))

In [ ]:
print(confusion_matrix(y_test,rfc_pred))

## Question 6:  Which model perfomed better?  Single Decision Tree or Random Forest? How can you tell?

# The end!